In [1]:
import pyxem as pxm
import numpy as np
import os
import gc
from pyxem.signals.diffraction_vectors import DiffractionVectors
import matplotlib as plt
%matplotlib qt

In [2]:
data_origin_folder = 'F:\\em18488_milled_pharma_sed\\raw\\paracetamol'

destination_folder = "E:\\hl585\\duncan_data_processed\\em18488_milled_pharma_sed\\paracetamol\\"
print(data_origin_folder)
print(destination_folder)

F:\em18488_milled_pharma_sed\raw\paracetamol
E:\hl585\duncan_data_processed\em18488_milled_pharma_sed\paracetamol\


In [4]:
filenames = []
print(data_origin_folder)
for root, dirs, files in os.walk(data_origin_folder):
    for file in files:
        if file.endswith(".mib"):
             filenames.append(os.path.join(root, file))

filenames = np.sort(filenames)

print(filenames)

F:\em18488_milled_pharma_sed\raw\paracetamol
['F:\\em18488_milled_pharma_sed\\raw\\paracetamol\\20180616 160517\\pca_200kX_15cm_A3-55.mib'
 'F:\\em18488_milled_pharma_sed\\raw\\paracetamol\\20180616 160750\\pca_200kX_15cm_A3-55.mib'
 'F:\\em18488_milled_pharma_sed\\raw\\paracetamol\\20180616 160952\\pca_200kX_15cm_A3-55.mib'
 'F:\\em18488_milled_pharma_sed\\raw\\paracetamol\\20180616 161158\\pca_200kX_15cm_A3-55.mib'
 'F:\\em18488_milled_pharma_sed\\raw\\paracetamol\\20180616 161412\\pca_200kX_15cm_A3-55.mib'
 'F:\\em18488_milled_pharma_sed\\raw\\paracetamol\\20180616 161800\\pca_200kX_15cm_A3-55.mib'
 'F:\\em18488_milled_pharma_sed\\raw\\paracetamol\\20180616 162001\\pca_200kX_15cm_A3-55.mib'
 'F:\\em18488_milled_pharma_sed\\raw\\paracetamol\\20180616 172906\\pca_200kX_15cm_A3-55.mib'
 'F:\\em18488_milled_pharma_sed\\raw\\paracetamol\\20180616 173218\\pca_200kX_15cm_A3-55.mib'
 'F:\\em18488_milled_pharma_sed\\raw\\paracetamol\\20180616 173505\\pca_200kX_15cm_A3-55.mib'
 'F:\\em18488_m

In [5]:
#just to figure out how to label the string when saving images
fname = filenames[5]
print(file[47:-4])
print(fname)
fname_label = fname[44:-5]


F:\em18488_milled_pharma_sed\raw\paracetamol\20180616 161800\pca_200kX_15cm_A3-55.mib


In [6]:
    # Load the data
    dp = pxm.load_mib(fname, reshape=True)
    dp.compute()
    print('finished dp.compute')
    #dp.plot(cmap='inferno', vmax=10)

    # Find peaks and form diffracting pixels map
    pks = dp.find_peaks(method='find_peaks_stat',
                        min_sigma=1.,
                        max_sigma=5.,
                        sigma_ratio=2.5,
                        threshold=0.04,
                        overlap=0.5,
                        show_progressbar=False#,
                       #interactive=False
                       )
    print(pks)
    print('peaks found')


reshaping using flyback pixel
[########################################] | 100% Completed | 12min 48.3s
finished dp.compute


NotImplementedError: The method `find_peaks_stat` is not implemented. See documentation for available implementations.

In [78]:
pks.inav[129,126].data

IndexError: index 126 is out of bounds for axis 0 with size 1

In [65]:
pks= DiffractionVectors.from_peaks(pks, center=(126,126), calibration=0.08)

  0%|          | 0/64770 [00:00<?, ?it/s]

In [69]:
    xim = pks.get_diffracting_pixels_map(binary=False)
    print('got diffracting pixels map')
    xim.change_dtype('float32')
    #xim.save(fname[:-5] + '_xim.tif')
    xim.save(destination_folder + fname_label+ '_xim.png')
    print('diffraction map saved')

  0%|          | 0/64770 [00:00<?, ?it/s]

got diffracting pixels map
diffraction map saved


In [ ]:

    # Calculate average diffraction
    dpeg = dp.mean((0,1))
    dpeg.change_dtype('float32')
    #dpeg.save(fname[:-5] + '_dpeg.tif')
    dpeg.save(destination_folder + fname_label+ '_dpeg.tif')
    print('diffraction pattern saved')


In [74]:
    # Calculate sum image
    mask = np.invert(dp.get_direct_beam_mask(25))
    dp = dp * mask
    dp = pxm.signals.ElectronDiffraction2D(dp)
    im = dp.sum((2,3))
    im = im.as_signal2D((0,1))
    im.change_dtype('float32')
    #im.save(fname[:-5] + '_im.tif')
    im.save(destination_folder + fname_label+ '_im.png')
    print('got sum intensity image')

    gc.collect()

got sum intensity image


12304